In [ ]:
import tkinter as tk
from tkinter import filedialog, scrolledtext, ttk, messagebox
from tkinter.font import Font
import os
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import json

# Import required packages with error handling
try:
    from nltk import download
    from sentence_transformers import SentenceTransformer, util
    from tabulate import tabulate
    nltk_available = True
except ImportError:
    nltk_available = False
    print("Warning: NLTK or sentence_transformers not available. Install with pip.")

# Global variables for model and data
model = None
ideal_data = None
current_filepath = None
selected_file_var = None
proceed_btn = None

def run_gui():
    global selected_file_var, proceed_btn
    
    # Create the main window
    root = tk.Tk()
    root.title("AI-Based Answer Evaluation System")
    root.geometry("900x600")
    root.configure(bg="#f5f5f5")
    
    # Make window open maximize
    root.state('zoomed')  # For Windows
  
    
    #  styles and fonts
    title_font = Font(family="Helvetica", size=24, weight="bold")
    subtitle_font = Font(family="Helvetica", size=16)
    body_font = Font(family="Helvetica", size=12)
    button_font = Font(family="Helvetica", size=12, weight="bold")
    
    # Main application frame
    main_frame = tk.Frame(root, bg="#f5f5f5")
    main_frame.pack(fill=tk.BOTH, expand=True)
    
    # Variables to store state
    selected_file_var = tk.StringVar()
    selected_file_var.set("No file selected")
    
    # Initialize model function - integrated from evaluation code
    def initialize_model():
        global model
        if not nltk_available:
            messagebox.showerror("Missing Dependencies", 
                               "Please install required packages: pip install nltk sentence-transformers")
            return False
        
        try:
            # Download NLTK data
            download('punkt')
            # Load the model
            model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
            return True
        except Exception as e:
            messagebox.showerror("Model Initialization Error", f"Error: {str(e)}")
            return False

    # Semantic similarity function - integrated from evaluation code
    def get_semantic_similarity(a1, a2):
        if not a1.strip() or not a2.strip():
            return 0.0
        emb1 = model.encode(a1, convert_to_tensor=True)
        emb2 = model.encode(a2, convert_to_tensor=True)
        return float(util.pytorch_cos_sim(emb1, emb2)[0][0])

    # Load correct answers function - integrated with proper error handling
    def load_correct_answers():
        global ideal_data
        
        try:
            # Ask user to locate the correct answers file
            filepath = filedialog.askopenfilename(
                title="Select Correct Answers JSON File",
                filetypes=[("JSON Files", "*.json")]
            )
            
            if not filepath:
                messagebox.showerror("Error", "Correct answers file is required.")
                return False
                
            with open(filepath, 'r', encoding='utf-8') as f:
                ideal_data = json.load(f)
            return True
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load correct answers: {str(e)}")
            return False

    # Function to process student answers - integrated with evaluation logic
    def process_student_answers(filepath):
        try:
            # For text files, convert to JSON format
            if filepath.lower().endswith('.txt'):
                student_data = convert_txt_to_json(filepath)
            # For JSON, load directly
            elif filepath.lower().endswith('.json'):
                with open(filepath, 'r', encoding='utf-8') as f:
                    student_data = json.load(f)
            # For images (future implementation)
            elif filepath.lower().endswith(('.png', '.jpg', '.jpeg')):
                messagebox.showinfo("Image Processing", 
                                 "Image processing would convert handwritten answers to text.")
                # Placeholder - return demo data for now
                return get_demo_results()
            else:
                messagebox.showerror("Error", "Unsupported file format")
                return None
                
            # Now evaluate the answers using our integration
            return evaluate_answers(student_data)
            
        except Exception as e:
            messagebox.showerror("Processing Error", f"Error: {str(e)}")
            return None

    # Function to convert text files to JSON format - integrated properly
    def convert_txt_to_json(filepath):
        # This is a simple implementation - modify based on your txt format
        student_data = {}
        current_qid = None
        current_answer = []
        
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Check if this is a question ID line (e.g., "Q1:" or "Question 1:")
            if line.lower().startswith(('q', 'question')) and ':' in line:
                # Save previous question if any
                if current_qid is not None and current_answer:
                    student_data[current_qid] = {"answer": ' '.join(current_answer)}
                    
                # Extract new question ID
                parts = line.split(':', 1)
                qid_text = parts[0].strip().lower()
                
                # Extract numeric part
                import re
                num_match = re.search(r'\d+', qid_text)
                if num_match:
                    current_qid = str(num_match.group())
                    current_answer = []
                    
                    # If there's content after the colon, add it to answer
                    if len(parts) > 1 and parts[1].strip():
                        current_answer.append(parts[1].strip())
            else:
                # Add to current answer
                if current_qid is not None:
                    current_answer.append(line)
        
        # Add the last question
        if current_qid is not None and current_answer:
            student_data[current_qid] = {"answer": ' '.join(current_answer)}
            
        return student_data

    # Evaluation function - integrated from evaluation code
    def evaluate_answers(student_data):
        if not ideal_data:
            messagebox.showerror("Error", "Correct answers not loaded")
            return None
            
        results = []
        total_score = 0
        max_score_per_question = 5
        max_score = len(ideal_data) * max_score_per_question
        
        # Evaluate each answer using semantic similarity
        for qid, ideal in ideal_data.items():
            student = student_data.get(qid, {"answer": ""})
            ideal_ans = ideal["answer"]
            student_ans = student["answer"]
            
            sem_sim = get_semantic_similarity(ideal_ans, student_ans)
            score = round(sem_sim * max_score_per_question, 2)  # scale to 5
            total_score += score
            
            results.append({
                "id": qid,
                "similarity": sem_sim,
                "score": score
            })
        
        # Calculate percentage and grade
        percentage = round((total_score / max_score) * 100, 2)
        
        # Determine grade
        if percentage >= 90:
            grade = "A+"
        elif percentage >= 80:
            grade = "A"
        elif percentage >= 70:
            grade = "B"
        elif percentage >= 60:
            grade = "C"
        elif percentage >= 50:
            grade = "D"
        else:
            grade = "F"
        
        return {
            "total_score": round(total_score, 2),
            "max_score": max_score,
            "percentage": percentage,
            "grade": grade,
            "questions": results
        }
    
    
    
    # FILE HANDLING FUNCTIONS
    def select_file():
        filetypes = [
            ("All Supported Files", "*.txt;*.png;*.jpg;*.jpeg;*.json"),
            ("Text Files", "*.txt"),
            ("JSON Files", "*.json"),
            ("Image Files", "*.png;*.jpg;*.jpeg")
        ]
        
        filepath = filedialog.askopenfilename(title="Select Answer Sheet", filetypes=filetypes)
        
        if filepath:
            filename = os.path.basename(filepath)
            selected_file_var.set(filename)
            
            # Enable proceed button
            proceed_btn.config(state=tk.NORMAL)
            
            # Store filepath
            global current_filepath
            current_filepath = filepath

    def process_and_show_results():
        # Show loading indicator
        progress_window = tk.Toplevel(root)
        progress_window.title("Processing")
        progress_window.geometry("300x100")
        progress_window.transient(root)
        
        progress_label = tk.Label(progress_window, text="Processing answers...\nPlease wait.", pady=20)
        progress_label.pack()
        
        progress_window.update()
        
        # Initialize model if not already done
        global model
        if model is None:
            if not initialize_model():
                progress_window.destroy()
                return
        
        # Load correct answers if not already loaded
        global ideal_data
        if ideal_data is None:
            if not load_correct_answers():
                progress_window.destroy()
                return
        
        # Process the student answers
        result_data = process_student_answers(current_filepath)
        
        # Close progress window
        progress_window.destroy()
        
        if result_data:
            create_results_page(result_data)
        else:
            messagebox.showerror("Error", "Failed to process answers.")

    # Function to get demo results for testing
    def get_demo_results():
        return {
            "total_score": 21,
            "max_score": 25,
            "percentage": 84,
            "grade": "A",
            "questions": [
                {"id": "1", "similarity": 0.89, "score": 4.5},
                {"id": "2", "similarity": 0.76, "score": 3.8},
                {"id": "3", "similarity": 0.92, "score": 4.6},
                {"id": "4", "similarity": 0.67, "score": 3.3},
                {"id": "5", "similarity": 0.82, "score": 4.1}
            ]
        }

    # PAGE 1: WELCOME PAGE
    def create_welcome_page():
        # Clear existing widgets
        for widget in main_frame.winfo_children():
            widget.destroy()
        
        # Welcome page content
        welcome_box = tk.Frame(main_frame, bg="white", bd=2, relief=tk.GROOVE)
        welcome_box.place(relx=0.5, rely=0.5, anchor=tk.CENTER, relwidth=0.8, relheight=0.8)
        
        # Try to load a logo
        try:
            logo_path = r"C:\Users\om sharma\OneDrive\Desktop\Screenshot 2025-04-23 215339.png"
            logo_img = Image.open(logo_path)
            logo_img = logo_img.resize((150, 150), Image.LANCZOS)
            logo_photo = ImageTk.PhotoImage(logo_img)
            logo_label = tk.Label(welcome_box, image=logo_photo, bg="white")
            logo_label.image = logo_photo  # Keep a reference
            logo_label.pack(pady=(30, 0))
        except Exception as e:
            # Fallback if logo not found
            print(f"Logo loading error: {e}")
            logo_label = tk.Label(welcome_box, text="ITM", font=Font(family="Arial", size=48, weight="bold"), bg="white", fg="#333")
            logo_label.pack(pady=(30, 0))
        
        # Project title
        title_label = tk.Label(welcome_box, text="AI-Based Answer Evaluation System", 
                             font=title_font, bg="white", fg="#333")
        title_label.pack(pady=(20, 10))
        
        # Project description
        description_frame = tk.Frame(welcome_box, bg="white", bd=0)
        description_frame.pack(pady=20)
        
        description_text = """
        An AI-powered system to evaluate handwritten answers with precision and speed.
        Leveraging semantic similarity using BERT to ensure fair and accurate assessment.
        """
        
        description_label = tk.Label(description_frame, text=description_text, 
                                  font=subtitle_font, bg="white", fg="#333",
                                  justify=tk.CENTER)
        description_label.pack()
        
        # Team details frame at bottom right
        team_frame = tk.Frame(welcome_box, bg="white")
        team_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=20, pady=20)
        
        # Team details
        team_label = tk.Label(team_frame, text="Project Team:", 
                           font=Font(family="Helvetica", size=12, weight="bold"), 
                           bg="white", fg="#333", anchor=tk.W)
        team_label.pack(side=tk.LEFT, anchor=tk.NW)
        
        # Create a frame for team members list
        members_frame = tk.Frame(team_frame, bg="white")
        members_frame.pack(side=tk.LEFT, padx=(10, 0))
        
        # Add team members as a list
        team_members = ["Om Sharma", "Animesh Arnav", "Mayuri Verma", "Deeksha Sharma"]
        for member in team_members:
            member_label = tk.Label(members_frame, text="• " + member,
                                 font=body_font, bg="white", fg="#333", anchor=tk.W)
            member_label.pack(anchor=tk.W)
        
        # Next button
        next_button = tk.Button(welcome_box, text="Next", font=button_font, 
                              bg="#4a86e8", fg="white", padx=30, pady=10,
                              relief=tk.FLAT, command=create_process_page)
        next_button.pack(side=tk.BOTTOM, pady=30)
    
    # PAGE 2: PROCESS FLOW PAGE
    def create_process_page():
        # Clear existing widgets
        for widget in main_frame.winfo_children():
            widget.destroy()
        
        # Process page content
        process_box = tk.Frame(main_frame, bg="white", bd=2, relief=tk.GROOVE)
        process_box.place(relx=0.5, rely=0.5, anchor=tk.CENTER, relwidth=0.8, relheight=0.8)
        
        # Title
        title_label = tk.Label(process_box, text="How It Works", 
                             font=title_font, bg="white", fg="#333")
        title_label.pack(pady=(30, 20))
        
        # Process flow content
        flow_frame = tk.Frame(process_box, bg="white", bd=0)
        flow_frame.pack(fill=tk.BOTH, expand=True, padx=50, pady=10)
        
        flow_text = """
✅ Handwriting Recognition: Uploaded documents (text or image) are converted 
   into readable text using Google Cloud Vision API.

🧠 Semantic Analysis: Uses BERT (Bidirectional Encoder Representations from 
   Transformers) to understand the meaning of your answer.

📊 Cosine Similarity: Compares your answer with the ideal one to measure 
   how closely they match in context and meaning.

🧾 Score Calculation: Scores are scaled based on similarity and compiled 
   into a result table showing per-question performance.

This ensures a fair and intelligent evaluation, focusing on how well 
the concept is understood rather than exact wording.
        """
        
        flow_textbox = scrolledtext.ScrolledText(flow_frame, 
                                              wrap=tk.WORD, 
                                              font=body_font,
                                              bg="white", 
                                              fg="#333",
                                              bd=0,
                                              height=15)
        flow_textbox.pack(fill=tk.BOTH, expand=True)
        flow_textbox.insert(tk.END, flow_text)
        flow_textbox.config(state=tk.DISABLED)
        
        # Navigation buttons
        buttons_frame = tk.Frame(process_box, bg="white")
        buttons_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=20, pady=20)
        
        # Back button
        back_button = tk.Button(buttons_frame, text="Back", font=body_font,
                              bg="#f0f0f0", fg="#333", padx=20, pady=8,
                              relief=tk.FLAT, command=create_welcome_page)
        back_button.pack(side=tk.LEFT)
        
        # Next button
        next_button = tk.Button(buttons_frame, text="Next", font=button_font,
                              bg="#4a86e8", fg="white", padx=30, pady=10,
                              relief=tk.FLAT, command=create_upload_page)
        next_button.pack(side=tk.RIGHT)
    
    # PAGE 3: UPLOAD PAGE
    def create_upload_page():
        global proceed_btn
        # Clear existing widgets
        for widget in main_frame.winfo_children():
            widget.destroy()
        
        # Upload page content
        upload_box = tk.Frame(main_frame, bg="white", bd=2, relief=tk.GROOVE)
        upload_box.place(relx=0.5, rely=0.5, anchor=tk.CENTER, relwidth=0.8, relheight=0.8)
        
        # Title
        title_label = tk.Label(upload_box, text="Upload Answer Sheet", 
                             font=title_font, bg="white", fg="#333")
        title_label.pack(pady=(30, 20))
        
        # Instructions
        instruction_label = tk.Label(upload_box, 
                                   text="Upload answer sheets in text or image format (both accepted)",
                                   font=subtitle_font, bg="white", fg="#333")
        instruction_label.pack(pady=(0, 10))
        
        # File types info
        filetypes_label = tk.Label(upload_box, 
                                 text="Supported file types: .txt, .png, .jpg, .jpeg, .json",
                                 font=body_font, bg="white", fg="#666")
        filetypes_label.pack(pady=(0, 20))
        
        # Upload area frame
        upload_area = tk.Frame(upload_box, bg="#f9f9f9", bd=1, relief=tk.SOLID)
        upload_area.pack(padx=100, pady=10, fill=tk.BOTH, expand=True)
        
        # Upload icon (text as placeholder)
        upload_icon_label = tk.Label(upload_area, text="📄", font=Font(family="Arial", size=48), bg="#f9f9f9")
        upload_icon_label.pack(pady=(20, 10))
        
        # Browse button
        browse_button = tk.Button(upload_area, text="Browse Files", font=body_font,
                                bg="#4a86e8", fg="white", padx=20, pady=5,
                                relief=tk.FLAT, command=select_file)
        browse_button.pack(pady=(5, 10))
        
        # Selected file label
        file_label = tk.Label(upload_area, textvariable=selected_file_var, 
                             font=body_font, bg="#f9f9f9", fg="#666")
        file_label.pack(pady=(0, 20))
        
        # Navigation buttons
        buttons_frame = tk.Frame(upload_box, bg="white")
        buttons_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=20, pady=20)
        
        # Back button
        back_button = tk.Button(buttons_frame, text="Back", font=body_font,
                              bg="#f0f0f0", fg="#333", padx=20, pady=8,
                              relief=tk.FLAT, command=create_process_page)
        back_button.pack(side=tk.LEFT)
        
        # Proceed Button (initially disabled)
        proceed_btn = tk.Button(buttons_frame, text="Proceed", font=button_font,
                              bg="#4a86e8", fg="white", padx=30, pady=10,
                              relief=tk.FLAT, command=process_and_show_results,
                              state=tk.DISABLED)
        proceed_btn.pack(side=tk.RIGHT)
        
        
        
        
        
        
        # PAGE 4: RESULTS PAGE - Integrated with evaluation results
    def create_results_page(result_data):
        # Clear existing widgets
        for widget in main_frame.winfo_children():
            widget.destroy()
        
        # Results page content
        results_box = tk.Frame(main_frame, bg="white", bd=2, relief=tk.GROOVE)
        results_box.place(relx=0.5, rely=0.5, anchor=tk.CENTER, relwidth=0.9, relheight=0.9)
        
        # Title
        title_label = tk.Label(results_box, text="Evaluation Results", 
                             font=title_font, bg="white", fg="#333")
        title_label.pack(pady=(20, 30))
        
        # Top section (Summary and Charts)
        top_frame = tk.Frame(results_box, bg="white")
        top_frame.pack(fill=tk.X, padx=20, pady=10)
        
        # Summary box
        summary_box = tk.Frame(top_frame, bg="white", bd=1, relief=tk.SOLID)
        summary_box.pack(side=tk.LEFT, padx=10, pady=10, fill=tk.BOTH)
        
        # Grade badge
        grade_frame = tk.Frame(summary_box, bg="#4a86e8", bd=0)
        grade_frame.pack(side=tk.RIGHT, padx=20)
        grade_label = tk.Label(grade_frame, text=result_data["grade"], 
                            font=Font(family="Arial", size=36, weight="bold"), 
                            bg="#4a86e8", fg="white", padx=15, pady=15)
        grade_label.pack()
        
        # Summary content
        summary_title = tk.Label(summary_box, text="Performance Summary", 
                               font=subtitle_font, bg="white", fg="#333")
        summary_title.pack(pady=(15, 10), padx=20, anchor=tk.W)
        
        # Summary data
        summary_data = [
            f"Total Score: {result_data['total_score']} out of {result_data['max_score']}",
            f"Percentage: {result_data['percentage']}%",
            f"Grade: {result_data['grade']}"
        ]
        
        for data in summary_data:
            data_label = tk.Label(summary_box, text=data, font=body_font, 
                                bg="white", fg="#333", anchor=tk.W)
            data_label.pack(pady=5, padx=20, anchor=tk.W)
        
        # Visualizations frame
        visualizations_frame = tk.Frame(top_frame, bg="white", bd=1, relief=tk.SOLID)
        visualizations_frame.pack(side=tk.RIGHT, padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Create charts - wrap in try block in case matplotlib isn't available
        try:
            create_charts(visualizations_frame, result_data)
        except Exception as e:
            error_label = tk.Label(visualizations_frame, 
                                text=f"Chart visualization unavailable: {str(e)}",
                                font=body_font, bg="white", fg="red", padx=20, pady=20)
            error_label.pack(expand=True)
        
        # Results table - improved implementation
        table_frame = tk.Frame(results_box, bg="white", bd=1, relief=tk.SOLID)
        table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Table title
        table_title = tk.Label(table_frame, text="Question-wise Analysis", 
                             font=subtitle_font, bg="white", fg="#333")
        table_title.pack(pady=(15, 10))
        
        # Create the improved results table
        create_results_table(table_frame, result_data)
        
        # Bottom buttons frame
        buttons_frame = tk.Frame(results_box, bg="white")
        buttons_frame.pack(fill=tk.X, padx=20, pady=20)
        
        # Download Report button
        download_button = tk.Button(buttons_frame, text="Download Report", font=button_font,
                                  bg="#4a86e8", fg="white", padx=20, pady=8,
                                  relief=tk.FLAT, command=download_report)
        download_button.pack(side=tk.RIGHT)
        
        # Back to Home button
        home_button = tk.Button(buttons_frame, text="Back to Home", font=body_font,
                              bg="#f0f0f0", fg="#333", padx=20, pady=8,
                              relief=tk.FLAT, command=create_welcome_page)
        home_button.pack(side=tk.LEFT)
    
    # Function to create the enhanced results table with proper layout
    def create_results_table(parent, result_data):
        # Create an enhanced table with proper scrolling
        table_container = tk.Frame(parent, bg="white")
        table_container.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Add scrollbar for the table
        table_scroll_frame = tk.Frame(table_container, bg="white")
        table_scroll_frame.pack(fill=tk.BOTH, expand=True)
        
        # Add horizontal and vertical scrollbars
        v_scrollbar = tk.Scrollbar(table_scroll_frame, orient=tk.VERTICAL)
        v_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        h_scrollbar = tk.Scrollbar(table_scroll_frame, orient=tk.HORIZONTAL)
        h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Create Treeview for table with scrollbars
        tree = ttk.Treeview(table_scroll_frame, columns=("qid", "similarity", "score", "performance"), 
                          show="headings", selectmode="none",
                          yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
        
        # Configure scrollbars
        v_scrollbar.config(command=tree.yview)
        h_scrollbar.config(command=tree.xview)
        
        # Style configuration for the table
        style = ttk.Style()
        style.configure("Treeview", 
                      background="white", 
                      foreground="#333", 
                      rowheight=30, 
                      fieldbackground="white",
                      font=body_font)
        
        style.configure("Treeview.Heading", 
                      font=Font(family="Helvetica", size=12, weight="bold"),
                      background="#4a86e8", 
                      foreground="white")
        
        style.map("Treeview", background=[("selected", "#e0e0e0")])
        
        # Define columns
        tree.heading("qid", text="Question ID")
        tree.heading("similarity", text="Semantic Similarity")
        tree.heading("score", text="Score")
        tree.heading("performance", text="Performance")
        
        # Column widths
        tree.column("qid", width=150, anchor=tk.CENTER)
        tree.column("similarity", width=150, anchor=tk.CENTER)
        tree.column("score", width=150, anchor=tk.CENTER)
        tree.column("performance", width=200, anchor=tk.CENTER)
        
        # Insert data rows
        for i, item in enumerate(result_data["questions"]):
            # Determine performance text and color based on score
            if item['score'] >= 4.5:
                perf_text = "Excellent"
                perf_color = "#28a745"  # Green
            elif item['score'] >= 3.5:
                perf_text = "Good"
                perf_color = "#17a2b8"  # Blue
            elif item['score'] >= 2.5:
                perf_text = "Average"
                perf_color = "#ffc107"  # Yellow
            else:
                perf_text = "Needs Improvement"
                perf_color = "#dc3545"  # Red
                
            # Insert with tag for row styling
            tag_name = f"row{i}"
            tree.insert("", tk.END, values=(
                f"Question {item['id']}", 
                f"{item['similarity']:.2f}", 
                f"{item['score']}/5", 
                perf_text
            ), tags=(tag_name,))
            
            # Configure tag with color
            tree.tag_configure(tag_name, background="#ffffff" if i % 2 == 0 else "#f9f9f9")
            
        # Pack the treeview
        tree.pack(fill=tk.BOTH, expand=True)
    
    # Function to create enhanced charts
    def create_charts(parent, result_data):
        # Create a frame to hold multiple charts
        charts_frame = tk.Frame(parent, bg="white")
        charts_frame.pack(fill=tk.BOTH, expand=True)
        
        # Left side for pie chart
        pie_frame = tk.Frame(charts_frame, bg="white")
        pie_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Right side for bar chart
        bar_frame = tk.Frame(charts_frame, bg="white")
        bar_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Data for charts
        scores = [item['score'] for item in result_data['questions']]
        question_ids = [f"Q{item['id']}" for item in result_data['questions']]
        similarities = [item['similarity'] for item in result_data['questions']]
        
        # Create pie chart
        pie_fig = plt.Figure(figsize=(4, 3.5), dpi=100)
        pie_ax = pie_fig.add_subplot(111)
        explode = [0.1 if score == max(scores) else 0 for score in scores]
        
        pie_ax.pie(scores, labels=question_ids, explode=explode, autopct='%1.1f%%', 
              shadow=True, startangle=90, colors=plt.cm.Paired(range(len(scores))))
        pie_ax.set_title('Score Distribution')
        
        # Create pie chart canvas
        pie_canvas = FigureCanvasTkAgg(pie_fig, master=pie_frame)
        pie_canvas.draw()
        pie_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=5)
        
        # Create bar chart for semantic similarity comparison
        bar_fig = plt.Figure(figsize=(4, 3.5), dpi=100)
        bar_ax = bar_fig.add_subplot(111)
        
        bar_colors = plt.cm.viridis(similarities)
        bars = bar_ax.bar(question_ids, similarities, color=bar_colors)
        
        # Add labels and formatting
        bar_ax.set_title('Semantic Similarity by Question')
        bar_ax.set_ylabel('Similarity Score')
        bar_ax.set_ylim(0, 1.0)
        bar_ax.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Add value labels on top of bars
        for bar in bars:
            height = bar.get_height()
            bar_ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=9)
        
        # Create bar chart canvas
        bar_canvas = FigureCanvasTkAgg(bar_fig, master=bar_frame)
        bar_canvas.draw()
        bar_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=5)
        
        # Progress bar for overall percentage
        progress_frame = tk.Frame(parent, bg="white")
        progress_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=20, pady=20)
        
        # Progress label
        progress_label = tk.Label(progress_frame, text=f"Overall: {result_data['percentage']}%", 
                             font=Font(family="Helvetica", size=12, weight="bold"), bg="white", fg="#333")
        progress_label.pack(anchor=tk.W, pady=(0, 5))
        
        # Progress bar with custom styling
        style = ttk.Style()
        style.configure("Custom.Horizontal.TProgressbar", 
                      background='#4a86e8', troughcolor='#f0f0f0', 
                      bordercolor='#f0f0f0', lightcolor='#4a86e8', 
                      darkcolor='#4a86e8')
        
        progress_bar = ttk.Progressbar(progress_frame, orient=tk.HORIZONTAL, 
                                   length=300, mode='determinate', 
                                   style="Custom.Horizontal.TProgressbar")
        progress_bar.pack(fill=tk.X)
        progress_bar["value"] = result_data['percentage']
    
    # Function to download report (placeholder)
    def download_report():
        try:
            # Ask user where to save the report
            filepath = filedialog.asksaveasfilename(
                defaultextension=".txt",
                filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")],
                title="Save Evaluation Report"
            )
            
            if not filepath:
                return  # User cancelled
                
            # Get the current result data from memory
            with open(filepath, "w", encoding="utf-8") as f:
                f.write("AI-BASED ANSWER EVALUATION REPORT\n")
                f.write("===============================\n\n")
                
                # Summary information
                if 'percentage' in result_data:
                    f.write(f"Overall Score: {result_data['total_score']:.2f}/{result_data['max_score']}\n")
                    f.write(f"Percentage: {result_data['percentage']}%\n")
                    f.write(f"Grade: {result_data['grade']}\n\n")
                
                # Question details
                f.write("QUESTION-WISE ANALYSIS\n")
                f.write("=====================\n\n")
                
                for item in result_data["questions"]:
                    f.write(f"Question {item['id']}:\n")
                    f.write(f"  Semantic Similarity: {item['similarity']:.2f}\n")
                    f.write(f"  Score: {item['score']}/5\n")
                    
                    # Performance level
                    if item['score'] >= 4.5:
                        perf_text = "Excellent"
                    elif item['score'] >= 3.5:
                        perf_text = "Good"
                    elif item['score'] >= 2.5:
                        perf_text = "Average"
                    else:
                        perf_text = "Needs Improvement"
                    
                    f.write(f"  Performance: {perf_text}\n\n")
                
                # Footer
                f.write("\n---\n")
                f.write("Generated by AI-Based Answer Evaluation System\n")
            
            messagebox.showinfo("Success", f"Report saved successfully to {filepath}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save report: {str(e)}")

    # Start with the welcome page
    create_welcome_page()
    
    # Run the application
    root.mainloop()

# Global references
model = None
ideal_data = None

# Run the GUI
if __name__ == "__main__":
    try:
        run_gui()
    except Exception as e:
        print(f"Error running application: {e}")

[nltk_data] Downloading package punkt to C:\Users\om
[nltk_data]     sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
